In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from keras.utils import pad_sequences

In [6]:
df = pd.read_csv('C:/Users/User/MLProjectTrails/grocery_ratings.csv')
df['date'] = pd.to_datetime(df['date'], unit='s')
df.sort_values(by=['reviewerID', 'date'], inplace=True)

In [9]:
user_groups = df.groupby('reviewerID')

X_train1, y_train1 = [], []
i = 0

for user, user_data in user_groups:
    user_data = user_data.sort_values(by='date')
    
    if len(user_data) > 5:
        i += 1
        product_sequence = user_data['product_id'].values
        rating_sequence = user_data['rating'].values
        
        X_train1.append((product_sequence[:5], rating_sequence[:5]))
        
        y_train1.append((product_sequence[5:], rating_sequence[5:]))
    
    if i % 100 == 0:
        print(f"Processed {i} users")

Processed 0 users
Processed 0 users
Processed 0 users
Processed 0 users
Processed 0 users
Processed 0 users
Processed 0 users
Processed 100 users
Processed 100 users
Processed 100 users
Processed 200 users
Processed 300 users
Processed 400 users
Processed 500 users
Processed 500 users
Processed 600 users
Processed 700 users
Processed 800 users
Processed 900 users
Processed 1000 users
Processed 1100 users
Processed 1200 users
Processed 1300 users
Processed 1300 users
Processed 1300 users
Processed 1400 users
Processed 1500 users
Processed 1600 users
Processed 1700 users
Processed 1700 users
Processed 1700 users
Processed 1800 users
Processed 1900 users
Processed 2000 users
Processed 2100 users
Processed 2200 users
Processed 2200 users
Processed 2300 users
Processed 2400 users
Processed 2500 users
Processed 2600 users
Processed 2700 users
Processed 2800 users
Processed 2800 users
Processed 2900 users
Processed 3000 users
Processed 3100 users
Processed 3200 users
Processed 3300 users
Proc

In [11]:
X_train = np.array(X_train1)

In [15]:
import numpy as np
def padsequences(sequences, max_length):
    """Pads sequences with 0 to a specified maximum length."""
    padded_sequences = []
    for seq in sequences:
        padded_seq = np.pad(seq, (0, max_length - len(seq)), 'constant', constant_values=0)  # Pad with 0
        padded_sequences.append(padded_seq)
    return np.array(padded_sequences)

max_len_products = max([len(seq[0]) for seq in y_train1])
max_len_ratings = max([len(seq[1]) for seq in y_train1])

y_train1_products = padsequences([seq[0] for seq in y_train1], max_len_products)
y_train1_ratings = padsequences([seq[1] for seq in y_train1], max_len_ratings)

y_train = np.array(list(zip(y_train1_products, y_train1_ratings)))

In [17]:
ratings_data = X_train[:, 1] 
X_train_padded = tensorflow.keras.utils.pad_sequences(ratings_data, maxlen=5, padding='post')

In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

product_ids = df['product_id'].unique()
product_id_map = {product_id: i+1 for i, product_id in enumerate(product_ids)}

X_train_mapped = [[product_id_map[prod] for prod in x[0]] for x in X_train]

X_train_padded = pad_sequences(X_train_mapped, maxlen=5)

y_train = np.array([y[1][0] for y in y_train], dtype=np.float32)  

model = Sequential()
model.add(Embedding(input_dim=len(product_id_map) + 1, output_dim=50, input_length=5))  
model.add(LSTM(128, activation='relu'))
model.add(Dense(1))  

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

model.fit(X_train_padded, y_train, epochs=2, batch_size=32)


Epoch 1/2
3542/3542 [==============================] - 628s 176ms/step - loss: 1.4123 - mae: 0.8724
Epoch 2/2
3542/3542 [==============================] - 631s 178ms/step - loss: 1.0961 - mae: 0.7563


In [22]:
predicted_ratings = model.predict(X_train_padded)

mae = mean_absolute_error(y_train,predicted_ratings)
print(mae)


3542/3542 [==============================] - 31s 9ms/step
0.61212504
